In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sqlalchemy import create_engine
from shapely import Point, wkt

import rasterio
import rioxarray
import json
from glob import glob

In [3]:
#Define path and one file example
PATH_WIO = "/Users/xavierpivan/HubOcean/wiosym/products/v2.1/output_geotiffs/"
PATH_SENSITIVITY = "/Users/xavierpivan/HubOcean/wiosym/"

sensitivity_path = PATH_SENSITIVITY+"Sensitivity Matrix 20220224.xlsx"

ecosystem_path = PATH_WIO+"eco/"
pressure_path = PATH_WIO+"pres/"

eco_files_list = glob(ecosystem_path+'*.tif')
pres_files_list = glob(pressure_path+'*.tif')

In [4]:
with open('/Users/xavierpivan/.secret.json', 'r') as file:
    conn_params = json.load(file)
conn_str = f"{conn_params['username']}:{conn_params['password']}@{conn_params['host']}/{conn_params['database']}"
engine = create_engine(conn_str)

In [15]:
# Format the connection string
xds = rioxarray.open_rasterio(eco_files_list[0])
lon = xds.x.values
lat = xds.y.values
#del xds

lonmin, lonmax = np.min(lon), np.max(lon)
latmin, latmax = np.min(lat), np.max(lat)

lon_grid, lat_grid = np.meshgrid(lon, lat)

print(f"Longitude ranges from {lonmin} to {lonmax}")
print(f"Latitude ranges from {latmin} to {latmax}")

Longitude ranges from 7.909201804826568 to 78.08112180482657
Latitude ranges from -40.04344441182884 to 16.08055558817116


In [75]:
query = f'''
    select id,
    ST_AsText(geom) AS geom_text,
    ST_XMin(ST_Envelope(geom)) AS min_lon,
    ST_YMin(ST_Envelope(geom)) AS min_lat,
    ST_XMax(ST_Envelope(geom)) AS max_lon,
    ST_YMax(ST_Envelope(geom)) AS max_lat,
    ST_Area(geom) AS area, 
    ST_Perimeter(geom) AS perimeter, 
    ST_AsText(ST_Centroid(geom)) AS centroid_text, 
    "ORIG_NAME"
    FROM marine_protected_areas
    WHERE ST_Within(
        ST_SetSRID(ST_Centroid(geom), 4326),
        ST_MakeEnvelope({lonmin}, {latmin}, {lonmax}, {latmax}, 4326)
    );
'''

In [80]:
df = pd.read_sql_query(query, engine)
df.set_index('id', inplace=True)
df['geometry'] = df['geom_text'].apply(wkt.loads)
df.drop(columns='geom_text', inplace=True)
multi_polygon = df['geometry'].iloc[0]

In [37]:
min_lon, min_lat, max_lon, max_lat = ex[['min_lon', 'min_lat', 'max_lon', 'max_lat']].values

In [60]:
top_left_corner = xds.sel(x=min_lon, y=max_lat, method='nearest')
bottom_right_corner = xds.sel(x=max_lon, y=min_lat, method='nearest')

ind_lon_0 = np.where(lon==top_left_corner.x.values)[0][0]
ind_lat_0 = np.where(lat==top_left_corner.y.values)[0][0]
ind_lon_1 = np.where(lon==bottom_right_corner.x.values)[0][0]
ind_lat_1 = np.where(lat==bottom_right_corner.y.values)[0][0]

lon_grid, lat_grid = np.meshgrid(lon[ind_lon_0:ind_lon_1], lat[ind_lat_0:ind_lat_1]) 

point_list = np.array(list(zip(np.ravel(lon_grid), np.ravel(lat_grid))))

In [82]:
from shapely.geometry import Point
import geopandas as gpd

points = [Point(x,y) for x, y in point_list]
gdf = gpd.GeoDataFrame({'geometry': [multi_polygon]})
points_gdf = gpd.GeoDataFrame(geometry=points)
within = gpd.sjoin(points_gdf, gdf, op='within')
indices = within.index.to_list()
point_symphony = point_list[indices]

/Users/xavierpivan/HubOcean/VesselEmissions/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


3 POINT (73.45440180482657 3.8945555881711584)
4 POINT (73.46358180482657 3.8945555881711584)
5 POINT (73.47276180482658 3.8945555881711584)
8 POINT (73.47276180482658 3.88555558817116)


In [88]:
x_list = [point for point in points if multi_polygon.contains(point)]

In [101]:
type(within)

geopandas.geodataframe.GeoDataFrame

In [ ]:
dx = lon[1]-lon[0]
dy = lat[1]-lat[0]

In [ ]:
from shapely.wkt import loads
from shapely.geometry import MultiPolygon, Polygon, Point

# Convert string representations to Shapely Point objects
points = [loads(p) for p in poly['vertex']]

polygon = Polygon([(p.x, p.y) for p in points])
multi_polygon = MultiPolygon([polygon])

In [34]:
len(lon)

7645

In [ ]:
poly

In [36]:
for index, row in df.iterrows():
    row
    

,id,min_lon,min_lat,max_lon,max_lat,area,perimeter,centroid_text,ORIG_NAME,geometry
0,1751,73.451318,3.880997,73.481495,3.899827,0.000306,0.077866,POINT(73.4678224716562 3.89205380542491),Guraidhoo Kandu,"MULTIPOLYGON (((73.455031515 3.89982694700007,..."
1,4814,29.295410,-31.910592,29.399243,-31.810610,0.003887,0.288456,POINT(29.3471218935452 -31.8608099950625),Hluleka Marine Protected Area,MULTIPOLYGON (((29.3222735050001 -31.810610202...
2,4815,28.825605,-32.417694,29.061070,-32.187983,0.027762,0.661598,POINT(28.9410922323297 -32.3014206221442),Dwesa-Cwebe Marine Protected Area,MULTIPOLYGON (((28.9369944580001 -32.187982653...
3,5142,18.250002,-34.407400,18.556883,-33.901249,0.091171,2.163697,POINT(18.3764684184988 -34.2038011629693),Table Mountain National Park Marine Protected ...,MULTIPOLYGON (((18.397447587 -33.9056472739999...
4,5143,30.726969,-30.373500,31.033300,-30.116663,0.060086,1.007017,POINT(30.9104352703796 -30.2494226440954),Aliwal Shoal Marine Protected Area,"MULTIPOLYGON (((31.0333000000001 -30.116663, 3..."
5,5144,29.552141,-31.695000,30.290003,-31.111555,0.128942,1.893814,POINT(29.9282874311571 -31.4196353001543),Pondoland Marine Protected Area,MULTIPOLYGON (((30.1773357440001 -31.111555101...
6,5776,18.896083,-34.407500,18.937339,-34.356652,0.001859,0.174206,POINT(18.9176194066879 -34.3848049273508),Betty's Bay Marine Protected Area,"MULTIPOLYGON (((18.937326828 -34.356651719, 18..."
7,5777,20.365421,-34.583161,20.927538,-34.453056,0.026435,1.171965,POINT(20.6187989749497 -34.5039264336271),De Hoop Marine Protected Area,"MULTIPOLYGON (((20.68281746 -34.453056325, 20...."
8,5778,22.835442,-34.103386,22.999923,-34.046413,0.001813,0.350347,POINT(22.9292482349497 -34.0820877149573),Goukamma Marine Protected Area,"MULTIPOLYGON (((22.958082195 -34.083251945, 22..."
9,5779,18.763540,-34.103337,18.810249,-34.081238,0.000162,0.103935,POINT(18.7913341998886 -34.0932565970758),Helderberg Marine Protected Area,"MULTIPOLYGON (((18.810248691 -34.099699024, 18..."
